In [ ]:
import os
from typing import Tuple, List, Callable, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.signal import correlate
import pydicom
import cv2

from PIL import Image

# **Functions for Core CT Image Processing**

The following functions are extracted from the core_ct package to handle CT scan image processing:

### Data Loading
- `load_dicom_files`: Loads DICOM files from a directory and returns 3D volume data along with pixel spacing and slice thickness information

### Slice Operations  
- `get_slice`: Extracts a 2D slice from the 3D volume along a specified axis
- `trim_slice`: Trims empty space from a slice based on an intensity threshold

### Brightness Analysis
- `get_brightness_trace`: Calculates mean brightness along a specified axis
- `get_brightness_stats`: Calculates brightness mean and standard deviation along an axis
- `process_brightness_data`: Processes CT scan data to get masked brightness and standard deviation statistics

### Visualization
- `display_slice`: Shows a single slice with optional physical dimensions
- `display_slice_bt_std`: Displays a core slice with corresponding brightness trace and standard deviation plots

In [ ]:
def load_dicom_files(dir_path: str, force: bool = True) -> Tuple[np.ndarray, float, float, float]:
    """
    Load DICOM files from a directory and return the 3D volume data.
    
    Args:
        dir_path: Path to directory containing DICOM files
        force: If True, ignore files that produce errors
    
    Returns:
        Tuple of (volume_data, pixel_spacing_x, pixel_spacing_y, slice_thickness)
    """
    # Get list of files in directory
    files = sorted([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
    
    slices = []
    for f in files:
        try:
            ds = pydicom.dcmread(os.path.join(dir_path, f))
            slices.append(ds)
        except:
            if not force:
                raise
            continue
    
    # Sort slices by ImagePositionPatient z coordinate
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    
    # Get image dimensions and spacing info
    pixel_spacing = slices[0].PixelSpacing
    slice_thickness = float(slices[0].SliceThickness)
    
    # Create 3D numpy array
    img_shape = list(slices[0].pixel_array.shape)
    img_shape.append(len(slices))
    volume_data = np.zeros(img_shape)
    
    # Fill 3D array with the images from the files
    for i, s in enumerate(slices):
        volume_data[:,:,i] = s.pixel_array
        
    return volume_data, pixel_spacing[0], pixel_spacing[1], slice_thickness

def get_slice(volume: np.ndarray, index: int, axis: int = 0) -> np.ndarray:
    """
    Extract a 2D slice from the 3D volume along specified axis.
    
    Args:
        volume: 3D numpy array of CT data
        index: Index of slice to extract
        axis: Axis along which to take slice (0, 1, or 2)
    
    Returns:
        2D numpy array of the slice
    """
    if axis == 0:
        return volume[index, :, :]
    elif axis == 1:
        return volume[:, index, :]
    else:
        return volume[:, :, index]

def trim_slice(slice_data: np.ndarray, threshold: float = 0.05) -> np.ndarray:
    """
    Trim empty space from a slice based on intensity threshold.
    
    Args:
        slice_data: 2D numpy array of slice data
        threshold: Value below which pixels are considered empty
        
    Returns:
        Trimmed 2D numpy array
    """
    # Find non-empty rows and columns
    row_mask = np.any(slice_data > threshold, axis=1)
    col_mask = np.any(slice_data > threshold, axis=0)
    
    # Get indices of non-empty regions
    row_indices = np.where(row_mask)[0]
    col_indices = np.where(col_mask)[0]
    
    # Trim the slice
    return slice_data[row_indices[0]:row_indices[-1]+1, 
                     col_indices[0]:col_indices[-1]+1]

def get_brightness_trace(slice_data: np.ndarray, axis: int = 1) -> np.ndarray:
    """
    Calculate brightness trace along specified axis.
    
    Args:
        slice_data: 2D numpy array of slice data
        axis: Axis along which to calculate mean (0 for vertical, 1 for horizontal)
    
    Returns:
        1D numpy array of mean values
    """
    return np.mean(slice_data, axis=axis)

def display_slice(slice_data: np.ndarray, 
                 pixel_spacing: Optional[Tuple[float, float]] = None,
                 title: str = "Core Slice") -> None:
    """
    Display a slice with optional physical dimensions.
    
    Args:
        slice_data: 2D numpy array of slice data
        pixel_spacing: Optional tuple of (x, y) pixel spacing in mm
        title: Plot title
    """
    plt.figure(figsize=(10, 6))
    
    if pixel_spacing is not None:
        extent = [0, slice_data.shape[1] * pixel_spacing[0],
                 0, slice_data.shape[0] * pixel_spacing[1]]
        plt.imshow(slice_data, extent=extent)
        plt.xlabel("Distance")
        plt.ylabel("Distance")
    else:
        plt.imshow(slice_data)
        plt.xlabel("Pixels")
        plt.ylabel("Pixels")
    
    plt.colorbar(label="Intensity")
    plt.title(title)
    plt.show()

def get_brightness_stats(slice_data: np.ndarray, axis: int = 1, width_start_pct: float = 0.25, width_end_pct: float = 0.75) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculate brightness mean and standard deviation along specified axis.
    
    Args:
        slice_data: 2D numpy array of slice data
        axis: Axis along which to calculate stats (0 for vertical, 1 for horizontal)
        width_start_pct: Starting percentage of width for the strip (default 0.25)
        width_end_pct: Ending percentage of width for the strip (default 0.75)
    
    Returns:
        Tuple of (mean values array, standard deviation array)
    """
    # Calculate the start and end indices based on provided percentages
    width = slice_data.shape[1]
    start_idx = int(width * width_start_pct)
    end_idx = int(width * width_end_pct)
    
    # Use only the specified strip for calculations
    center_slice = slice_data[:, start_idx:end_idx]
    
    return np.mean(center_slice, axis=axis), np.std(center_slice, axis=axis)

def display_slice_bt_std(slice_data: np.ndarray,
                        brightness: np.ndarray,
                        stddev: np.ndarray,
                        pixel_spacing: Optional[Tuple[float, float]] = None,
                        core_name: str = "",
                        save_figs: bool = False,
                        output_dir: Optional[str] = None) -> None:
    """
    Display a core slice and corresponding brightness trace and standard deviation.
    Similar to core_ct's display_slice_bt_std function.
    
    Args:
        slice_data: 2D numpy array of slice data
        brightness: 1D array of mean brightness values
        stddev: 1D array of standard deviation values
        pixel_spacing: Optional tuple of (x, y) pixel spacing in mm
        core_name: Name of the core to display in title
        save_figs: Whether to save figures to files (default False)
        output_dir: Directory to save figures if save_figs is True
    """
    # Create figure with 3 subplots with specific width ratios and smaller space between subplots
    # Calculate height based on data dimensions while keeping width fixed at 8
    height = 2 * (slice_data.shape[0] / slice_data.shape[1])
    fig, (ax1, ax2, ax3) = plt.subplots(
        1, 3, 
        figsize=(7, height),
        sharey=True,
        gridspec_kw={'width_ratios': [1.7, 0.6, 0.3], 'wspace': 0.22}
    )
    # Plot the slice
    slice_dim = slice_data.shape
    if pixel_spacing is not None:
        extent = [
            0, slice_dim[1] * pixel_spacing[0],
            slice_dim[0] * pixel_spacing[1], 0
        ]  # Note: y-axis inverted
        im = ax1.imshow(slice_data, extent=extent, cmap='jet', vmin=400, vmax=1700)  # vmin set to the defined minimum brightness threshold
        ax1.set_xlabel("Width", fontsize='small')
        ax1.set_ylabel("Depth", fontsize='small')
    else:
        im = ax1.imshow(slice_data, cmap='jet', vmin=400, vmax=1700)  # vmin set to the defined minimum brightness threshold
        ax1.set_xlabel("Width (pixels)", fontsize='small')
        ax1.set_ylabel("Depth (pixels)", fontsize='small')
    
    # Add colorbar with small tick labels
    cbar = plt.colorbar(im, ax=ax1)
    cbar.ax.tick_params(labelsize='x-small')
    
    # Calculate y coordinates for brightness and stddev plots
    y_coords = np.arange(len(brightness))
    if pixel_spacing is not None:
        y_coords = y_coords * pixel_spacing[1]
    
    # Plot brightness trace with standard deviation shading
    ax2.plot(brightness, y_coords, 'b-', label='Mean')
    ax2.fill_betweenx(
        y_coords, 
        brightness - stddev, 
        brightness + stddev, 
        alpha=0.1, color='b', label='±1σ', linewidth=0
    )
    ax2.set_xlabel("CT# ±1σ", fontsize='small')
    ax2.grid(True)
    ax2.set_xlim(left=400)  # Set x-axis to start at 400
    
    # Plot standard deviation
    ax3.plot(stddev, y_coords)
    ax3.set_xlabel("σ (STDEV)", fontsize='small')
    ax3.grid(True)
    
    # Add text annotation if core name is provided
    if core_name:
        fig.text(.5, 0.92, core_name, fontweight='bold', ha='center', va='top')
    
    # Make tick labels smaller for all axes
    for ax in [ax1, ax2, ax3]:
        ax.tick_params(axis='both', which='major', labelsize='x-small')
    
    # show plot
    plt.show()

    # Save figures if requested
    if save_figs:
        if output_dir is None:
            raise ValueError("output_dir must be provided when save_figs is True")
            
        # Save full figure as PNG and SVG
        output_file = os.path.join(output_dir, f"{core_name}.png")
        fig.savefig(output_file, dpi=300, bbox_inches='tight')
        print(f"Composite CT results saved to: ~/{'/'.join(output_file.split('/')[-2:])}")
        
        output_file = os.path.join(output_dir, f"{core_name}.svg") 
        fig.savefig(output_file, bbox_inches='tight')
        print(f"Composite CT image saved to: ~/{'/'.join(output_file.split('/')[-2:])}")
        
        # Create and save colormap image as compressed TIFF
        # Calculate aspect ratio of data
        data_height, data_width = slice_data.shape
        data_aspect = data_height / data_width
        
        # Set figure size to maintain data aspect ratio while filling width
        fig_width = 2  # Keep original width
        fig_height = fig_width * data_aspect
        
        fig_img = plt.figure(figsize=(fig_width, fig_height), dpi=300)
        
        # Create axes that fills entire figure
        ax = plt.axes([0, 0, 1, 1])
        ax.set_axis_off()
        
        if pixel_spacing is not None:
            im_img = plt.imshow(slice_data, extent=extent, cmap='jet', vmin=400, aspect='auto')
        else:
            im_img = plt.imshow(slice_data, cmap='jet', vmin=400, aspect='auto')
            
        # Ensure no padding
        plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
        plt.margins(0,0)
        
        # Convert matplotlib figure to RGB array
        fig_img.canvas.draw()
        img_array = np.array(fig_img.canvas.renderer.buffer_rgba())[:,:,:3]
        
        # Save as compressed TIFF using PIL
        output_file = os.path.join(output_dir, f"{core_name}.tiff")
        img = Image.fromarray(img_array)
        img.save(output_file, format='TIFF', compression='tiff_deflate')
        print(f"CT image saved to: ~/{'/'.join(output_file.split('/')[-2:])}")
        plt.close(fig_img)


def process_brightness_data(slice_data, px_spacing_y, trim_top, trim_bottom, min_brightness=400, buffer=5, width_start_pct=0.25, width_end_pct=0.75):
    """
    Process CT scan slice data to get brightness and standard deviation statistics with masking.
    
    Args:
        slice_data: 2D numpy array of CT scan slice data
        px_spacing_y: Pixel spacing in y direction (mm/pixel)
        trim_top: Amount to trim from top in mm
        trim_bottom: Amount to trim from bottom in mm 
        buffer_mm: Buffer size in mm around masked values (default 5)
        min_brightness: Minimum brightness threshold (default 400)
        width_start_pct: Starting percentage of width to use for brightness calculation (default 0.25)
        width_end_pct: Ending percentage of width to use for brightness calculation (default 0.75)
        
    Returns:
        brightness: Masked brightness values
        stddev: Masked standard deviation values
    """
    # Trim the slice based on trim values
    if trim_top == 0 and trim_bottom == 0:
        trimmed_slice = trim_slice(slice_data)
    elif trim_top == 0:
        trimmed_slice = trim_slice(slice_data[:-trim_bottom])
    elif trim_bottom == 0:
        trimmed_slice = trim_slice(slice_data[trim_top:])
    else:
        trimmed_slice = trim_slice(slice_data[trim_top:-trim_bottom])
    
    # Get brightness stats with width percentage parameters
    brightness, stddev = get_brightness_stats(trimmed_slice, width_start_pct=width_start_pct, width_end_pct=width_end_pct)
    
    # Convert buffer from mm to pixels
    px_per = 1/px_spacing_y
    buffer_px = int(buffer * px_per)
    
    # Find indices below minimum brightness
    neg_indices = np.where(brightness < min_brightness)[0]
    
    # Create buffer zones around values below threshold
    buffer_zones = []
    for idx in neg_indices:
        start = max(0, idx - buffer_px)
        end = min(len(brightness), idx + buffer_px + 1)
        buffer_zones.extend(range(start, end))
        
    # Create and apply mask
    mask = np.ones(len(brightness), dtype=bool)
    mask[buffer_zones] = False
    brightness[~mask] = np.nan
    stddev[~mask] = np.nan
    
    return brightness, stddev, trimmed_slice

# **Functions for Curve Matching and Stitching**

The following functions handle matching and stitching of CT scan brightness curves:

### Overlap Detection
- `find_best_overlap`: Finds optimal overlap between two curves by maximizing:
  1. Pearson correlation coefficient ($r$) between overlapping sections:
     $$r = \frac{\sum(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum(x_i - \bar{x})^2\sum(y_i - \bar{y})^2}}$$
  2. Peak matching score based on prominence alignment
     Final score: $score = r + \sum_{peaks} w_p$ where $w_p = 2.0$ per matched peak pair

### Curve Stitching  
- `stitch_curves`: Combines overlapping curves by:
  1. Finding optimal overlap position
  2. Averaging overlap region values: $v_{overlap} = \frac{v_1 + v_2}{2}$
  3. Concatenating sections: $v_{stitched} = [v_1[:-o], v_{overlap}, v_2[o:]]$
     where $o$ is overlap length

### Visualization
- `plot_stitched_curves`: Displays stitching results with:
  - Original curves and overlap regions
  - Final stitched curve
  - Stitch point markers
  - Brightness and standard deviation plots


In [ ]:
# Function to calculate correlation at different overlaps
def find_best_overlap(curve1, curve2, min_overlap=20, max_overlap=450):
    max_corr = -np.inf
    best_overlap = 0
    
    # Remove NaN values for correlation calculation
    curve1_clean = pd.Series(curve1).fillna(0)
    curve2_clean = pd.Series(curve2).fillna(0)
    
    # Find peaks with prominence to identify major peaks
    curve1_peaks, properties1 = scipy.signal.find_peaks(curve1_clean, prominence=50)
    curve2_peaks, properties2 = scipy.signal.find_peaks(curve2_clean, prominence=50)
    
    # Find valleys (negative peaks)
    curve1_valleys, properties1_valleys = scipy.signal.find_peaks(-curve1_clean, prominence=50)
    curve2_valleys, properties2_valleys = scipy.signal.find_peaks(-curve2_clean, prominence=50)
    
    # Sort peaks by prominence
    curve1_peak_heights = properties1['prominences']
    curve2_peak_heights = properties2['prominences']
    curve1_valley_depths = properties1_valleys['prominences']
    curve2_valley_depths = properties2_valleys['prominences']
    
    # Get first peak/valley and top peaks/valleys
    curve1_first_peak = curve1_peaks[0] if len(curve1_peaks) > 0 else None
    curve2_first_peak = curve2_peaks[0] if len(curve2_peaks) > 0 else None
    curve1_first_valley = curve1_valleys[0] if len(curve1_valleys) > 0 else None
    curve2_first_valley = curve2_valleys[0] if len(curve2_valleys) > 0 else None
    curve1_major_peaks = curve1_peaks[np.argsort(curve1_peak_heights)[-10:]] # Top 10 peaks
    curve2_major_peaks = curve2_peaks[np.argsort(curve2_peak_heights)[-10:]]
    curve1_major_valleys = curve1_valleys[np.argsort(curve1_valley_depths)[-5:]] # Top 5 valleys
    curve2_major_valleys = curve2_valleys[np.argsort(curve2_valley_depths)[-5:]]
    
    # Try different overlap positions
    for overlap in range(min_overlap, min(len(curve1), len(curve2), max_overlap)):
        # Get overlapping sections
        overlap1 = curve1_clean[-overlap:]
        overlap2 = curve2_clean[:overlap]
        
        if len(overlap1) == len(overlap2):
            # Calculate regular correlation
            corr = np.corrcoef(overlap1, overlap2)[0,1]
            
            # Calculate peak matching score with weighted priorities
            peak_score = 0
            
            # Check first peaks in overlap region
            if curve1_first_peak is not None and curve2_first_peak is not None:
                if curve1_first_peak >= len(curve1)-overlap:
                    p1_adjusted = curve1_first_peak - (len(curve1) - overlap)
                    if abs(p1_adjusted - curve2_first_peak) < 5:  # Small tolerance
                        peak_score += 2.0  # Highest weight for first peak match
            
            # Check first valleys in overlap region
            if curve1_first_valley is not None and curve2_first_valley is not None:
                if curve1_first_valley >= len(curve1)-overlap:
                    v1_adjusted = curve1_first_valley - (len(curve1) - overlap)
                    if abs(v1_adjusted - curve2_first_valley) < 5:  # Small tolerance
                        peak_score += 1.2  # High weight for first valley match
            
            # Check major peaks in overlap region
            overlap1_peaks = [p for p in curve1_major_peaks if p >= len(curve1)-overlap]
            overlap2_peaks = [p for p in curve2_major_peaks if p < overlap]
            
            # Add bonus for matching major peaks
            if len(overlap1_peaks) > 0 and len(overlap2_peaks) > 0:
                for p1 in overlap1_peaks:
                    for p2 in overlap2_peaks:
                        p1_adjusted = p1 - (len(curve1) - overlap)
                        if abs(p1_adjusted - p2) < 5:  # Small tolerance
                            peak_score += 0.5  # Weight for major peak matches
                            
            # Check major valleys in overlap region
            overlap1_valleys = [v for v in curve1_major_valleys if v >= len(curve1)-overlap]
            overlap2_valleys = [v for v in curve2_major_valleys if v < overlap]
            
            # Add bonus for matching major valleys
            if len(overlap1_valleys) > 0 and len(overlap2_valleys) > 0:
                for v1 in overlap1_valleys:
                    for v2 in overlap2_valleys:
                        v1_adjusted = v1 - (len(curve1) - overlap)
                        if abs(v1_adjusted - v2) < 3:  # Small tolerance
                            peak_score += 0.3  # Weight for major valley matches
            
            # Combine scores with emphasis on peaks first, then correlation
            total_score = peak_score + corr
            
            if total_score > max_corr:
                max_corr = total_score
                best_overlap = overlap
                
    return best_overlap, max_corr

def stitch_curves(brightness_1, brightness_2, stddev_1, stddev_2, px_spacing_y_1, px_spacing_y_2, min_overlap=20, max_overlap=450):
    # Convert to true depth values for overlap finding
    depth_1 = np.arange(len(brightness_1)) * px_spacing_y_1
    depth_2 = np.arange(len(brightness_2)) * px_spacing_y_2
    
    # Find best overlap position using only brightness
    final_overlap, brightness_corr = find_best_overlap(brightness_1, brightness_2, min_overlap=min_overlap, max_overlap=max_overlap)
    
    print(f"Brightness overlap: {final_overlap} pixels (correlation: {brightness_corr:.3f})")
    
    # Get overlapping sections in true depth
    overlap_depth_1 = np.arange(len(brightness_1)-final_overlap, len(brightness_1)) * px_spacing_y_1
    overlap_depth_2 = np.arange(final_overlap) * px_spacing_y_2  # Use curve 2's spacing
    
    # Calculate shift values based on the overlap region
    brightness_shift = np.nanmean(brightness_1[-final_overlap:] - brightness_2[:final_overlap])  #avoid NaN
    stddev_shift = np.nanmean(stddev_1[-final_overlap:] - stddev_2[:final_overlap])              #avoid NaN
    
    # Shift the entire second dataset
    brightness_2_shifted = brightness_2 + brightness_shift
    stddev_2_shifted = stddev_2 + stddev_shift
    
    print(f"Applied brightness shift: {brightness_shift:.3f}")
    print(f"Applied stddev shift: {stddev_shift:.3f}")
    
    # Create averaged values in overlap region using shifted brightness and stddev
    # Create masks for invalid values (NaN or zero)
    mask1_brightness = ~(np.isnan(brightness_1[-final_overlap:]) | (brightness_1[-final_overlap:] == 0))
    mask2_brightness = ~(np.isnan(brightness_2_shifted[:final_overlap]) | (brightness_2_shifted[:final_overlap] == 0))
    mask1_stddev = ~(np.isnan(stddev_1[-final_overlap:]) | (stddev_1[-final_overlap:] == 0))
    mask2_stddev = ~(np.isnan(stddev_2_shifted[:final_overlap]) | (stddev_2_shifted[:final_overlap] == 0))
    
    # Only average where both values are valid
    overlap_brightness = np.full(final_overlap, np.nan)
    overlap_stddev = np.full(final_overlap, np.nan)
    valid_mask_brightness = mask1_brightness & mask2_brightness
    valid_mask_stddev = mask1_stddev & mask2_stddev
    
    overlap_brightness[valid_mask_brightness] = (brightness_1[-final_overlap:][valid_mask_brightness] + 
                                               brightness_2_shifted[:final_overlap][valid_mask_brightness]) / 2
    overlap_stddev[valid_mask_stddev] = (stddev_1[-final_overlap:][valid_mask_stddev] + 
                                        stddev_2_shifted[:final_overlap][valid_mask_stddev]) / 2
    
    # Stitch the curves using the final overlap position with averaged overlap region
    stitched_brightness = np.concatenate([
        brightness_1[:-final_overlap],  # Non-overlapped part of curve 1
        overlap_brightness,             # Averaged overlap region
        brightness_2_shifted[final_overlap:]  # Non-overlapped part of curve 2
    ])
    stitched_stddev = np.concatenate([
        stddev_1[:-final_overlap],
        overlap_stddev,
        stddev_2_shifted[final_overlap:]
    ])
    
    # Create depth array for the stitched data using appropriate spacing for each section
    depth_before_overlap = np.arange(len(brightness_1)-final_overlap) * px_spacing_y_1
    depth_overlap = overlap_depth_1  # Use spacing from curve 1 for overlap region
    depth_after_overlap = (np.arange(len(brightness_2)-final_overlap) * px_spacing_y_2) + overlap_depth_1[-1]
    stitched_depth = np.concatenate([depth_before_overlap, depth_overlap, depth_after_overlap])
    
    return (final_overlap, overlap_depth_1, overlap_depth_2, 
            stitched_brightness, stitched_stddev, stitched_depth, 
            brightness_2_shifted, stddev_2_shifted)

def plot_stitched_curves(stitched_depth, stitched_brightness, stitched_stddev, 
                        brightness_1, brightness_2, stddev_1, stddev_2,
                        brightness_2_shifted, stddev_2_shifted,
                        final_overlap, overlap_depth_1, overlap_depth_2, px_spacing_y_1, px_spacing_y_2):
    # Plot the stitched results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 5))
    
    # Calculate overlap region correctly using overlap_depth_1
    overlap_start = overlap_depth_1[0]
    overlap_end = overlap_depth_1[-1]
    
    # Add shaded overlap region for brightness
    ax1.axvspan(overlap_start, overlap_end, 
                color='gray', alpha=0.2, label='Overlap Region')
    
    # Plot overlapping sections
    ax1.plot(overlap_depth_1, brightness_1[-final_overlap:], 'g:', label='Overlapped Curve 1')
    ax1.plot(overlap_depth_1, brightness_2_shifted[:final_overlap], 'r:', label='Adjusted Curve 2')
    
    # Plot brightness curves
    ax1.plot(stitched_depth, stitched_brightness, 'b-', label='Stitched Curves')
    
    # Create depth arrays for original and shifted curves using respective spacings
    depth_2 = np.arange(len(brightness_2)) * px_spacing_y_2
    depth_2_shifted = depth_2 + overlap_start  # Adjust to use overlap_start
    
    ax1.set_xlabel('Depth')
    ax1.set_ylabel('Adjusted CT#')
    ax1.grid(True)
    ax1.legend(loc='upper left', fontsize='x-small')
    
    # Add shaded overlap region for stddev
    ax2.axvspan(overlap_start, overlap_end, 
                color='gray', alpha=0.2, label='Overlap Region')
    
    # Plot overlapping sections
    ax2.plot(overlap_depth_1, stddev_1[-final_overlap:], 'g:', label='Overlapped Curve 1')
    ax2.plot(overlap_depth_1, stddev_2_shifted[:final_overlap], 'r:', label='Adjusted Curve 2')
    
    # Plot standard deviation curves
    ax2.plot(stitched_depth, stitched_stddev, 'b-', label='Stitched Curves')
    ax2.set_xlabel('Depth')
    ax2.set_ylabel('Adjusted σ (STDEV)')
    ax2.grid(True)
    ax2.legend(loc='upper left', fontsize='x-small')
    
    plt.tight_layout()
    plt.show()

def create_stitched_slice(trimmed_slice_1, trimmed_slice_2, final_overlap, px_spacing_x_1, px_spacing_y_1, px_spacing_x_2, px_spacing_y_2):
    """
    Create a stitched CT slice from two overlapping slices.
    
    Parameters:
    -----------
    trimmed_slice_1 : ndarray
        First CT slice to stitch
    trimmed_slice_2 : ndarray 
        Second CT slice to stitch
    final_overlap : int
        Number of overlapping pixels between slices
    px_spacing_x_1 : float
        Pixel spacing in x direction for first slice
    px_spacing_y_1 : float
        Pixel spacing in y direction for first slice
    px_spacing_x_2 : float
        Pixel spacing in x direction for second slice
    px_spacing_y_2 : float
        Pixel spacing in y direction for second slice
        
    Returns:
    --------
    tuple:
        stitched_slice : ndarray
            The stitched CT slice
        pixel_spacing : tuple
            (x_spacing, y_spacing) for the stitched slice
    """
    # Get dimensions of both slices
    height1, width1 = trimmed_slice_1.shape
    height2, width2 = trimmed_slice_2.shape

    # Use the smaller width for both slices
    min_width = min(width1, width2)

    # Calculate new heights maintaining aspect ratio
    if width1 > width2:
        new_height1 = int(height1 * (min_width / width1))
        trimmed_slice_1 = cv2.resize(trimmed_slice_1, (min_width, new_height1))
    elif width2 > width1:
        new_height2 = int(height2 * (min_width / width2))
        trimmed_slice_2 = cv2.resize(trimmed_slice_2, (min_width, new_height2))

    stitched_slice = np.vstack([
        trimmed_slice_1,
        trimmed_slice_2[final_overlap:]
    ])

    # Calculate pixel spacing for stitched image - use average of both slices
    stitched_px_spacing = ((px_spacing_x_1 + px_spacing_x_2)/2, 
                          (px_spacing_y_1 + px_spacing_y_2)/2)
    
    return stitched_slice, stitched_px_spacing
####

## **Functions for CT data auto stitching**

In [ ]:
def process_single_scan(data_dir, params, segment, scan_name, width_start_pct=0.25, width_end_pct=0.75, max_value_side_trim=1200):
    """Process a single scan and return the processed data"""
    volume_data, px_spacing_x, px_spacing_y, slice_thickness = load_dicom_files(data_dir)
    slice_data = get_slice(volume_data, index=int(len(volume_data[0,0,:])*1/2), axis=2)
    
    # Trim left and right columns where all values are < max_value_side_trim (outside of the cores)
    left_idx = 0
    right_idx = slice_data.shape[1]
    
    for col in range(slice_data.shape[1]):
        if (slice_data[:,col] >= max_value_side_trim).any():
            left_idx = col
            break
            
    for col in range(slice_data.shape[1]-1, -1, -1):
        if (slice_data[:,col] >= max_value_side_trim).any():
            right_idx = col + 1
            break
    
    slice_data = slice_data[:, left_idx:right_idx]
    
    brightness, stddev, trimmed_slice = process_brightness_data(slice_data,
                                              px_spacing_y,
                                              trim_top=params['trim_top'],
                                              trim_bottom=params['trim_bottom'],
                                              min_brightness=params['min_brightness'],
                                              buffer=params['buffer'],
                                              width_start_pct=width_start_pct,
                                              width_end_pct=width_end_pct)
    
    display_slice_bt_std(trimmed_slice, brightness, stddev,
                        pixel_spacing=(px_spacing_x, px_spacing_y),
                        core_name=f"{segment} ({scan_name})")
    
    return brightness, stddev, trimmed_slice, px_spacing_x, px_spacing_y

def process_two_scans(segment_data, segment, mother_dir, width_start_pct=0.25, width_end_pct=0.75, max_value_side_trim=1200, min_overlap=20, max_overlap=450):
    """Process and stitch two scans together"""
    scans = segment_data['scans']
    data_dir_1 = f"{mother_dir}/{segment}/{scans[0]}"
    data_dir_2 = f"{mother_dir}/{segment}/{scans[1]}"
    
    # Process first scan
    bright1, std1, trimmed_slice1, px_spacing_x1, px_spacing_y1 = process_single_scan(
        data_dir_1, 
        segment_data['params'][scans[0]], 
        segment, 
        scans[0],
        width_start_pct=width_start_pct,
        width_end_pct=width_end_pct,
        max_value_side_trim=max_value_side_trim
    )
    
    # Process second scan
    bright2, std2, trimmed_slice2, px_spacing_x2, px_spacing_y2 = process_single_scan(
        data_dir_2,
        segment_data['params'][scans[1]],
        segment,
        scans[1],
        width_start_pct=width_start_pct,
        width_end_pct=width_end_pct,
        max_value_side_trim=max_value_side_trim
    )
    
    # Stitch scans
    final_overlap, od1, od2, st_bright, st_std, st_depth, bright2_shifted, std2_shifted = stitch_curves(
        bright1, bright2, std1, std2, px_spacing_y1, px_spacing_y2, min_overlap=min_overlap, max_overlap=max_overlap
    )
    
    # Plot and display results
    plot_stitched_curves(st_depth, st_bright, st_std,
                        bright1, bright2, std1, std2,
                        bright2_shifted, std2_shifted,
                        final_overlap, od1, od2, px_spacing_y1, px_spacing_y2)

    # Create stitched slice
    st_slice, pixel_spacing = create_stitched_slice(
        trimmed_slice1, trimmed_slice2, final_overlap,
        px_spacing_x1, px_spacing_y1, px_spacing_x2, px_spacing_y2
    )

    # Recalculate brightness and std dev on stitched slice without trimming
    st_bright_re, st_std_re, st_slice = process_brightness_data(st_slice, 
                                                               pixel_spacing[1],
                                                               trim_top=0,
                                                               trim_bottom=0,
                                                               min_brightness=400,
                                                               buffer=5,
                                                               width_start_pct=width_start_pct,
                                                               width_end_pct=width_end_pct)
    
    # Get depth array from length of recalculated brightness
    st_depth_re = np.arange(len(st_bright_re))
    
    # Display stitched slice with recalculated brightness/std
    display_slice_bt_std(st_slice, st_bright_re, st_std_re,
                        pixel_spacing=pixel_spacing,
                        core_name=f"{segment} (stitched)")
    
    return st_bright_re, st_std_re, st_depth_re, st_slice, pixel_spacing


In [ ]:
def process_and_stitch_segments(core_structure, mother_dir, width_start_pct=0.25, width_end_pct=0.75, max_value_side_trim=1200, min_overlap=20, max_overlap=450):
    # Dictionary to store stitched results for each core segment
    stitched_segments = {}
    
    # Process each segment
    for segment, segment_data in core_structure.items():
        print(f"Processing {segment}")
        
        if 'suffixes' not in segment_data:  # Regular segments with one or two scans
            if len(segment_data['scans']) == 1:  # Single scan
                data_dir = f"{mother_dir}/{segment}/{segment_data['scans'][0]}"
                brightness, stddev, trimmed_slice, px_spacing_x, px_spacing_y = process_single_scan(
                    data_dir,
                    segment_data['params'][segment_data['scans'][0]],
                    segment,
                    segment_data['scans'][0],
                    width_start_pct=width_start_pct,
                    width_end_pct=width_end_pct,
                    max_value_side_trim=max_value_side_trim
                )
                
                # Rescale slice to match rgb dimensions
                target_height = segment_data['rgb_pxlength']
                target_width = segment_data['rgb_pxwidth']
                
                rescaled_slice = cv2.resize(trimmed_slice, (target_width, target_height))
                
                # Interpolate brightness and stddev to match new height
                new_bright = np.interp(np.linspace(0, len(brightness)-1, target_height),
                                     np.arange(len(brightness)), brightness)
                new_std = np.interp(np.linspace(0, len(stddev)-1, target_height),
                                  np.arange(len(stddev)), stddev)
                
                stitched_segments[segment] = {
                    'brightness': new_bright,
                    'stddev': new_std,
                    'depth': np.arange(target_height),  # Use pixel units
                    'slice': rescaled_slice,
                    'px_spacing': (1, 1)  # Use pixel units
                }
                
                # If upside_down is True, rotate slice and reverse data arrays
                if segment_data.get('upside_down', False):
                    stitched_segments[segment]['slice'] = cv2.rotate(rescaled_slice, cv2.ROTATE_180)
                    stitched_segments[segment]['brightness'] = np.flip(new_bright)
                    stitched_segments[segment]['stddev'] = np.flip(new_std)
                
                # Display rescaled slice
                core_name = f"[UPSIDE DOWN] {segment} (rescaled to {target_height} x {target_width}px)" if segment_data.get('upside_down', False) else f"{segment} (rescaled to {target_height} x {target_width}px)"
                display_slice_bt_std(stitched_segments[segment]['slice'], stitched_segments[segment]['brightness'], stitched_segments[segment]['stddev'],
                                   pixel_spacing=(1, 1),
                                   core_name=core_name)
                
            else:  # Two scans
                st_bright, st_std, st_depth, st_slice, pixel_spacing = process_two_scans(
                    segment_data, segment, mother_dir, width_start_pct=width_start_pct, width_end_pct=width_end_pct, max_value_side_trim=max_value_side_trim, min_overlap=min_overlap, max_overlap=max_overlap
                )
                
                # Rescale slice to match rgb dimensions
                target_height = segment_data['rgb_pxlength']
                target_width = segment_data['rgb_pxwidth']
                
                rescaled_slice = cv2.resize(st_slice, (target_width, target_height))
                
                # Interpolate brightness and stddev to match new height
                new_bright = np.interp(np.linspace(0, len(st_bright)-1, target_height), 
                                     np.arange(len(st_bright)), st_bright)
                new_std = np.interp(np.linspace(0, len(st_std)-1, target_height),
                                  np.arange(len(st_std)), st_std)
                
                stitched_segments[segment] = {
                    'brightness': new_bright,
                    'stddev': new_std,
                    'depth': np.arange(target_height),  # Use pixel units
                    'slice': rescaled_slice,
                    'px_spacing': (1, 1)  # Use pixel units
                }
                
                # If upside_down is True, rotate slice and reverse data arrays
                if segment_data.get('upside_down', False):
                    stitched_segments[segment]['slice'] = cv2.rotate(rescaled_slice, cv2.ROTATE_180)
                    stitched_segments[segment]['brightness'] = np.flip(new_bright)
                    stitched_segments[segment]['stddev'] = np.flip(new_std)
                
                # Display rescaled slice
                core_name = f"[UPSIDE DOWN] {segment} (rescaled to {target_height} x {target_width}px)" if segment_data.get('upside_down', False) else f"{segment} (rescaled to {target_height} x {target_width}px)"
                display_slice_bt_std(stitched_segments[segment]['slice'], stitched_segments[segment]['brightness'], stitched_segments[segment]['stddev'],
                                   pixel_spacing=(1, 1),
                                   core_name=core_name)
                
        else:  # Segments with suffixes (A, B, C or A, B)
            print(f"Processing {segment}: Stitching sections {', '.join(segment_data['suffixes'])}")
            brightness_list = []
            stddev_list = []
            slice_list = []
            
            # Process each section
            for suffix in segment_data['suffixes']:
                folder_name = f"{segment}{suffix}/{segment_data['scans'][0]}"
                data_dir = f"{mother_dir}/{folder_name}"
                param_key = f"{suffix}/{segment_data['scans'][0]}"
                
                brightness, stddev, trimmed_slice, px_spacing_x, px_spacing_y = process_single_scan(
                    data_dir,
                    segment_data['params'][param_key],
                    f"{segment}{suffix}",
                    segment_data['scans'][0],
                    width_start_pct=width_start_pct,
                    width_end_pct=width_end_pct,
                    max_value_side_trim=max_value_side_trim
                )
                
                brightness_list.append(brightness)
                stddev_list.append(stddev)
                slice_list.append(trimmed_slice)
            
            if len(segment_data['suffixes']) == 1:  # Only A
                # Use data from single section without stitching
                st_bright = brightness_list[0]
                st_std = stddev_list[0]
                st_slice = slice_list[0]
                
                # Rescale slice to match rgb dimensions
                target_height = segment_data['rgb_pxlength']
                target_width = segment_data['rgb_pxwidth']
                
                rescaled_slice = cv2.resize(st_slice, (target_width, target_height))
                
                # Interpolate brightness and stddev to match new height
                new_bright = np.interp(np.linspace(0, len(st_bright)-1, target_height),
                                     np.arange(len(st_bright)), st_bright)
                new_std = np.interp(np.linspace(0, len(st_std)-1, target_height),
                                  np.arange(len(st_std)), st_std)
                
                stitched_segments[segment] = {
                    'brightness': new_bright,
                    'stddev': new_std,
                    'depth': np.arange(target_height),  # Use pixel units
                    'slice': rescaled_slice,
                    'px_spacing': (1, 1)  # Use pixel units
                }
                
                # If upside_down is True, rotate slice and reverse data arrays
                if segment_data.get('upside_down', False):
                    stitched_segments[segment]['slice'] = cv2.rotate(rescaled_slice, cv2.ROTATE_180)
                    stitched_segments[segment]['brightness'] = np.flip(new_bright)
                    stitched_segments[segment]['stddev'] = np.flip(new_std)
                
                # Display rescaled slice
                core_name = f"[UPSIDE DOWN] {segment} (rescaled to {target_height} x {target_width}px)" if segment_data.get('upside_down', False) else f"{segment} (rescaled to {target_height} x {target_width}px)"
                display_slice_bt_std(stitched_segments[segment]['slice'], stitched_segments[segment]['brightness'], stitched_segments[segment]['stddev'],
                                   pixel_spacing=(1, 1),
                                   core_name=core_name)
                
            elif len(segment_data['suffixes']) == 2:  # A and B
                # Stitch A and B
                final_overlap_ab, od1_ab, od2_ab, st_bright_ab, st_std_ab, st_depth_ab, bright2_shifted_ab, std2_shifted_ab = stitch_curves(
                    brightness_list[0], brightness_list[1], 
                    stddev_list[0], stddev_list[1],
                    px_spacing_y, px_spacing_y,  # Add second px_spacing_y parameter
                    min_overlap=min_overlap, max_overlap=max_overlap
                )
                
                st_slice_ab, pixel_spacing_ab = create_stitched_slice(
                    slice_list[0], slice_list[1],
                    final_overlap_ab, px_spacing_x, px_spacing_y, px_spacing_x, px_spacing_y
                )
                plot_stitched_curves(st_depth_ab, st_bright_ab, st_std_ab,
                                   brightness_list[0], brightness_list[1],
                                   stddev_list[0], stddev_list[1],
                                   bright2_shifted_ab, std2_shifted_ab,
                                   final_overlap_ab, od1_ab, od2_ab, px_spacing_y, px_spacing_y)
                
                # Rescale slice to match rgb dimensions
                target_height = segment_data['rgb_pxlength']
                target_width = segment_data['rgb_pxwidth']
                
                rescaled_slice = cv2.resize(st_slice_ab, (target_width, target_height))
                
                # Interpolate brightness and stddev to match new height
                new_bright = np.interp(np.linspace(0, len(st_bright_ab)-1, target_height),
                                     np.arange(len(st_bright_ab)), st_bright_ab)
                new_std = np.interp(np.linspace(0, len(st_std_ab)-1, target_height),
                                  np.arange(len(st_std_ab)), st_std_ab)
                
                stitched_segments[segment] = {
                    'brightness': new_bright,
                    'stddev': new_std,
                    'depth': np.arange(target_height),  # Use pixel units
                    'slice': rescaled_slice,
                    'px_spacing': (1, 1)  # Use pixel units
                }
                
                # If upside_down is True, rotate slice and reverse data arrays
                if segment_data.get('upside_down', False):
                    stitched_segments[segment]['slice'] = cv2.rotate(rescaled_slice, cv2.ROTATE_180)
                    stitched_segments[segment]['brightness'] = np.flip(new_bright)
                    stitched_segments[segment]['stddev'] = np.flip(new_std)
                
                # Display rescaled slice
                core_name = f"[UPSIDE DOWN] {segment} (rescaled to {target_height} x {target_width}px)" if segment_data.get('upside_down', False) else f"{segment} (rescaled to {target_height} x {target_width}px)"
                display_slice_bt_std(stitched_segments[segment]['slice'], stitched_segments[segment]['brightness'], stitched_segments[segment]['stddev'],
                                   pixel_spacing=(1, 1),
                                   core_name=core_name)
                
            else:  # A, B and C
                # Stitch A and B first
                final_overlap_ab, od1_ab, od2_ab, st_bright_ab, st_std_ab, st_depth_ab, bright2_shifted_ab, std2_shifted_ab = stitch_curves(
                    brightness_list[0], brightness_list[1], 
                    stddev_list[0], stddev_list[1],
                    px_spacing_y, px_spacing_y,  # Add second px_spacing_y parameter
                    min_overlap=min_overlap, max_overlap=max_overlap
                )
                
                st_slice_ab, pixel_spacing_ab = create_stitched_slice(
                    slice_list[0], slice_list[1],
                    final_overlap_ab, px_spacing_x, px_spacing_y, px_spacing_x, px_spacing_y
                )
                plot_stitched_curves(st_depth_ab, st_bright_ab, st_std_ab,
                                   brightness_list[0], brightness_list[1],
                                   stddev_list[0], stddev_list[1],
                                   bright2_shifted_ab, std2_shifted_ab,
                                   final_overlap_ab, od1_ab, od2_ab, px_spacing_y, px_spacing_y)
                
                # Stitch AB with C
                final_overlap_abc, od1_abc, od2_abc, st_bright, st_std, st_depth, bright2_shifted_abc, std2_shifted_abc = stitch_curves(
                    st_bright_ab, brightness_list[2],
                    st_std_ab, stddev_list[2],
                    px_spacing_y, px_spacing_y,  # Add second px_spacing_y parameter
                    min_overlap=min_overlap, max_overlap=max_overlap
                )
                
                st_slice, pixel_spacing = create_stitched_slice(
                    st_slice_ab, slice_list[2],
                    final_overlap_abc, px_spacing_x, px_spacing_y, px_spacing_x, px_spacing_y
                )
                
                # Rescale slice to match rgb dimensions
                target_height = segment_data['rgb_pxlength']
                target_width = segment_data['rgb_pxwidth']
                
                rescaled_slice = cv2.resize(st_slice, (target_width, target_height))
                
                # Interpolate brightness and stddev to match new height
                new_bright = np.interp(np.linspace(0, len(st_bright)-1, target_height),
                                     np.arange(len(st_bright)), st_bright)
                new_std = np.interp(np.linspace(0, len(st_std)-1, target_height),
                                  np.arange(len(st_std)), st_std)
                
                stitched_segments[segment] = {
                    'brightness': new_bright,
                    'stddev': new_std,
                    'depth': np.arange(target_height),  # Use pixel units
                    'slice': rescaled_slice,
                    'px_spacing': (1, 1)  # Use pixel units
                }
                
                plot_stitched_curves(st_depth, st_bright, st_std,
                                   st_bright_ab, brightness_list[2],
                                   st_std_ab, stddev_list[2],
                                   bright2_shifted_abc, std2_shifted_abc,
                                   final_overlap_abc, od1_abc, od2_abc, px_spacing_y, px_spacing_y)
                
                # If upside_down is True, rotate slice and reverse data arrays
                if segment_data.get('upside_down', False):
                    stitched_segments[segment]['slice'] = cv2.rotate(rescaled_slice, cv2.ROTATE_180)
                    stitched_segments[segment]['brightness'] = np.flip(new_bright)
                    stitched_segments[segment]['stddev'] = np.flip(new_std)
                
                # Display rescaled slice
                core_name = f"[UPSIDE DOWN] {segment} (rescaled to {target_height} x {target_width}px)" if segment_data.get('upside_down', False) else f"{segment} (rescaled to {target_height} x {target_width}px)"
                display_slice_bt_std(stitched_segments[segment]['slice'], stitched_segments[segment]['brightness'], stitched_segments[segment]['stddev'],
                                   pixel_spacing=(1, 1),
                                   core_name=core_name)
    
    # Stack all segments together (from top to bottom)
    segment_order = list(core_structure.keys())
    print(f"Stitching all core segments together: {', '.join(segment_order)}")
    final_slices = []
    final_brightness = []
    final_stddev = []
    cumulative_depth = 0
    
    # Find minimum width among all segments
    min_width = min([stitched_segments[seg]['slice'].shape[1] for seg in segment_order])

    for segment in segment_order:
        data = stitched_segments[segment]
        slice_data = data['slice']
        
        # Calculate scale factors
        current_width = slice_data.shape[1]
        width_scale = min_width / current_width
        
        # Rescale slice while preserving aspect ratio
        new_height = int(slice_data.shape[0] * width_scale)
        slice_data = cv2.resize(slice_data, (min_width, new_height))
        final_slices.append(slice_data)
        
        # Interpolate brightness and stddev to match new height
        old_indices = np.arange(len(data['brightness']))
        new_indices = np.linspace(0, len(data['brightness'])-1, new_height)
        
        new_brightness = np.interp(new_indices, old_indices, data['brightness'])
        new_stddev = np.interp(new_indices, old_indices, data['stddev'])
        
        # Adjust depths to be continuous
        depths = np.arange(new_height) + cumulative_depth
        cumulative_depth = depths[-1]
        
        final_brightness.extend(new_brightness)
        final_stddev.extend(new_stddev)

    # Create final stitched arrays
    final_stitched_slice = np.vstack(final_slices)
    final_stitched_brightness = np.array(final_brightness)
    final_stitched_stddev = np.array(final_stddev)
    final_stitched_depth = np.arange(len(final_stitched_brightness))  # Use pixel units
    
    return final_stitched_slice, final_stitched_brightness, final_stitched_stddev, final_stitched_depth, 1, 1  # Use pixel units


<hr>

### Define core CT data

### M9907-22PC CT data auto-stitiching & data csv table output

In [ ]:
# # Define core information
# cruise_name = "M9907"
# core_name = "22PC"
# total_length_cm = 501  # Adjust this value based on actual core length
# width_start_pct=0.25
# width_end_pct=0.75

# # Define base path
# mother_dir = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/CT_data_2022"

# # Define core segments and their scan folders. Order means from top to bottom
# core_structure = {
#     f'{cruise_name}-{core_name}-4': {
#         'scans': ['SE000000'],
#         'params': {
#             'SE000000': {'trim_top': 820, 'trim_bottom': 15, 'min_brightness': 400, 'buffer': 5}   #without cut off the T1 T2, trim_top = 30
#         },
#         'rgb_pxlength': 4555, 'rgb_pxwidth': 995,
#         'upside_down': False   #True if the CT image is upside down
#     },
#     f'{cruise_name}-{core_name}-3': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 55, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 10},
#             'SE000002': {'trim_top': 20, 'trim_bottom': 55, 'min_brightness': 400, 'buffer': 10}
#         },
#         'rgb_pxlength': 15050, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-2': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 55, 'trim_bottom': 110, 'min_brightness': 400, 'buffer': 10},
#             'SE000002': {'trim_top': 120, 'trim_bottom': 40, 'min_brightness': 400, 'buffer': 10}
#         },
#         'rgb_pxlength': 14970, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-1': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 45, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 1, 'trim_bottom': 35, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14890, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }
# }


### M9907-23PC CT data auto-stitiching

In [ ]:
# # Define core information
# cruise_name = "M9907"
# core_name = "23PC"
# total_length_cm = 783  # Adjust this value based on actual core length
# width_start_pct=0.25
# width_end_pct=0.75

# # Define base path
# mother_dir = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/CT_data_2022"

# # Define core segments and their scan folders for M9907-23PC
# core_structure = {
#     f'{cruise_name}-{core_name}-6': {
#         'scans': ['SE000000'], #without cut off the T0
#         # 'scans': ['SE000000', 'SE000002'], #to cut off the T0
#         'params': {
#             # 'SE000000': {'trim_top': 60, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 5}, #without cut off the T0
#             # 'SE000002': {'trim_top': 5, 'trim_bottom': 60, 'min_brightness': 400, 'buffer': 5}  #without cut off the T0
#             'SE000000': {'trim_top': 62, 'trim_bottom': 532, 'min_brightness': 400, 'buffer': 5}   #to cut off the T0
#         },
#         'rgb_pxlength': 6305, 'rgb_pxwidth': 995,
#         'upside_down': True
#     },
#     f'{cruise_name}-{core_name}-5': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 50, 'trim_bottom': 190, 'min_brightness': 400, 'buffer': 5},
#             'SE000002': {'trim_top': 150, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14965, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-4': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 50, 'trim_bottom': 150, 'min_brightness': 400, 'buffer': 5},
#             'SE000002': {'trim_top': 170, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14945, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-3': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 50, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 20, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 1, 'trim_bottom': 40, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14890, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-2': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 45, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 20, 'trim_bottom': 70, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 11185, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-1': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 65, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'SE000002': {'trim_top': 20, 'trim_bottom': 45, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14915, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }
# }


### M9907-25PC CT data auto-stitiching

In [ ]:
# # Define core information
# cruise_name = "M9907"
# core_name = "25PC"
# total_length_cm = 797  # Adjust this value based on actual core length
# width_start_pct=0.25
# width_end_pct=0.75

# # Define base path
# mother_dir = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/CT_data_2022"

# # Define core segments and their scan folders
# core_structure = {
#     f'{cruise_name}-{core_name}-6': {
#         # 'scans': ['SE000000', 'SE000002'], 
#         'scans': ['SE000002'], 
#         'params': {
#             # 'SE000000': {'trim_top': 60, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 50}, 
#             'SE000002': {'trim_top': 122, 'trim_bottom': 42, 'min_brightness': 400, 'buffer': 3}  
#         },
#         'rgb_pxlength': 7635, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-5': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 60, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 20, 'trim_bottom': 60, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 14958, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-4': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 40, 'trim_bottom': 180, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 20, 'trim_bottom': 45, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 14955, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-3': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 50, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 320, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 14860, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-2': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 80, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 230, 'trim_bottom': 40, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 11510, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-1': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 60, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 1},
#             'SE000002': {'trim_top': 100, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 1}
#         },
#         'rgb_pxlength': 14905, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }
# }


### M9907-11PC CT data auto-stitiching

In [ ]:
# # Define core information
# cruise_name = "M9907"
# core_name = "11PC"
# total_length_cm = 439  # Adjust this value based on actual core length
# width_start_pct=0.25
# width_end_pct=0.75

# # Define base path
# mother_dir = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/CT_data_2022"

# # Define core segments and their scan folders
# core_structure = {
#     f'{cruise_name}-{core_name}-3': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 53, 'trim_bottom': 120, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 480, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 13625, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-2': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 50, 'trim_bottom': 3, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 360, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 15055, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-1': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 40, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 1},
#             'SE000002': {'trim_top': 85, 'trim_bottom': 47, 'min_brightness': 400, 'buffer': 1}
#         },
#         'rgb_pxlength': 14885, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }
# }


### M9907-12PC CT data auto-stitiching

In [ ]:
# # Define core information
# cruise_name = "M9907"
# core_name = "12PC"
# total_length_cm = 488  # Adjust this value based on actual core length
# width_start_pct=0.25
# width_end_pct=0.75

# # Define base path
# mother_dir = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/CT_data_2022"

# # Define core segments and their scan folders
# core_structure = {
#     f'{cruise_name}-{core_name}-4': {
#         'scans': ['SE000000'],
#         'params': {
#             'SE000000': {'trim_top': 882, 'trim_bottom': 45, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 3535, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-3': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 45, 'trim_bottom': 3, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 230, 'trim_bottom': 47, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 14935, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-2': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 37, 'trim_bottom': 150, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 60, 'trim_bottom': 60, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 14845, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-1': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 47, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 1},
#             'SE000002': {'trim_top': 315, 'trim_bottom': 40, 'min_brightness': 400, 'buffer': 1}
#         },
#         'rgb_pxlength': 14870, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }
# }


### M9907-56PC CT data auto-stitiching

In [ ]:
# # Define core information
# cruise_name = "RR0207"
# core_name = "56PC"
# total_length_cm = 794  # Adjust this value based on actual core length
# width_start_pct=0.15 
# width_end_pct=0.85

# # Define base path
# mother_dir = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/CT_data_2022"

# # Define core segments and their scan folders (for RR0207-56PC, core number order is 1 -> 6)
# core_structure = {
#     f'{cruise_name}-{core_name}-1': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 40, 'trim_bottom': 200, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 150, 'trim_bottom': 30, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 14905, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-2': {
#         'suffixes': ['A', 'B'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 37, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 35, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 7885, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-3': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 35, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 1, 'trim_bottom': 60, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 15010, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-4': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 40, 'trim_bottom': 300, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 100, 'trim_bottom': 27, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 15075, 'rgb_pxwidth': 996,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-5': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 45, 'trim_bottom': 400, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 600, 'trim_bottom': 40, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 11355, 'rgb_pxwidth': 996,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-6': {
#         'scans': ['SE000000', 'SE000002'],  
#         'params': {
#             'SE000000': {'trim_top': 20, 'trim_bottom': 250, 'min_brightness': 400, 'buffer': 3}, 
#             'SE000002': {'trim_top': 200, 'trim_bottom': 30, 'min_brightness': 400, 'buffer': 3}  
#         },
#         'rgb_pxlength': 14900, 'rgb_pxwidth': 996,
#         'upside_down': False
#     }
# }


### M9907-14TC CT data auto-stitiching

In [ ]:
# # Define core information
# cruise_name = "M9907"
# core_name = "14TC"
# total_length_cm = 199  # Adjust this value based on actual core length
# width_start_pct=0.3
# width_end_pct=0.7

# # Define base path
# mother_dir = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/CT_data_2022"

# # Define core segments and their scan folders
# core_structure = {
#     f'{cruise_name}-{core_name}-2': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 60, 'trim_bottom':40, 'min_brightness': 450, 'buffer': 2},
#             'SE000002': {'trim_top': 300, 'trim_bottom': 42, 'min_brightness': 420, 'buffer': 2}
#         },
#         'rgb_pxlength': 14785, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-1': {
#         'scans': ['SE000000'],
#         'params': {
#             'SE000000': {'trim_top': 18, 'trim_bottom': 825, 'min_brightness': 390, 'buffer': 3}
#         },
#         'rgb_pxlength': 4705, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }
# }


### M9907-30PC CT data auto-stitiching

In [ ]:
# # Define core information
# cruise_name = "M9907"
# core_name = "30PC"
# total_length_cm = 781  # Adjust this value based on actual core length
# width_start_pct=0.25
# width_end_pct=0.75

# # Define base path
# mother_dir = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/CT_data_2024"

# # Define core segments and their scan folders
# core_structure = {
#     f'{cruise_name}-{core_name}-6': {
#         'suffixes': ['A', 'B'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 47, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 95, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 7050, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-5': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 50, 'trim_bottom': 10, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 20, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14975, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }, 
#     f'{cruise_name}-{core_name}-4': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 40, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 5, 'trim_bottom': 52, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 15055, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }, 
#     f'{cruise_name}-{core_name}-3': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 40, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 20, 'trim_bottom': 35, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14860, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }, 
#     f'{cruise_name}-{core_name}-2': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 40, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 150, 'trim_bottom': 85, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 11355, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }, 
#     f'{cruise_name}-{core_name}-1': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 45, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 5, 'trim_bottom': 290, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 13820, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }
# }


### M9907-31PC CT data auto-stitiching

In [ ]:
# Define core information
cruise_name = "M9907"
core_name = "31PC"
total_length_cm = 767  # Adjust this value based on actual core length
width_start_pct=0.2
width_end_pct=0.8

# Define base path
mother_dir = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/CT_data_2024"

# Define core segments and their scan folders
core_structure = {
    f'{cruise_name}-{core_name}-6': {
        'suffixes': ['A'],
        'scans': ['SE000000'],
        'params': {
            'A/SE000000': {'trim_top': 60, 'trim_bottom': 100, 'min_brightness': 400, 'buffer': 3}
        },
        'rgb_pxlength': 4215, 'rgb_pxwidth': 995,
        'upside_down': True
    },
    f'{cruise_name}-{core_name}-5': {
        'suffixes': ['A', 'B', 'C'],
        'scans': ['SE000000'],
        'params': {
            'A/SE000000': {'trim_top': 45, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
            'B/SE000000': {'trim_top': 1, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 3},
            'C/SE000000': {'trim_top': 5, 'trim_bottom': 65, 'min_brightness': 400, 'buffer': 3}
        },
        'rgb_pxlength': 14975, 'rgb_pxwidth': 995,
        'upside_down': False
    }, 
    f'{cruise_name}-{core_name}-4': {
        'suffixes': ['A', 'B', 'C'],
        'scans': ['SE000000'],
        'params': {
            'A/SE000000': {'trim_top': 49, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
            'B/SE000000': {'trim_top': 5, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 3},
            'C/SE000000': {'trim_top': 5, 'trim_bottom': 52, 'min_brightness': 400, 'buffer': 3}
        },
        'rgb_pxlength': 14930, 'rgb_pxwidth': 995,
        'upside_down': False
    }, 
    f'{cruise_name}-{core_name}-3': {
        'suffixes': ['A', 'B', 'C'],
        'scans': ['SE000000'],
        'params': {
            'A/SE000000': {'trim_top': 37, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
            'B/SE000000': {'trim_top': 1, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 3},
            'C/SE000000': {'trim_top': 5, 'trim_bottom': 45, 'min_brightness': 400, 'buffer': 3}
        },
        'rgb_pxlength': 14905, 'rgb_pxwidth': 995,
        'upside_down': False
    }, 
    f'{cruise_name}-{core_name}-2': {
        'suffixes': ['A', 'B', 'C'],
        'scans': ['SE000000'],
        'params': {
            'A/SE000000': {'trim_top': 45, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
            'B/SE000000': {'trim_top': 1, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 3},
            'C/SE000000': {'trim_top': 5, 'trim_bottom': 65, 'min_brightness': 400, 'buffer': 3}
        },
        'rgb_pxlength': 11430, 'rgb_pxwidth': 995,
        'upside_down': False
    }, 
    f'{cruise_name}-{core_name}-1': {
        'suffixes': ['A', 'B', 'C'],
        'scans': ['SE000000'],
        'params': {
            'A/SE000000': {'trim_top': 35, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
            'B/SE000000': {'trim_top': 1, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
            'C/SE000000': {'trim_top': 1, 'trim_bottom': 40, 'min_brightness': 400, 'buffer': 3}
        },
        'rgb_pxlength': 15025, 'rgb_pxwidth': 995,
        'upside_down': False
    }
}


<hr>

### Execution

In [ ]:
# Process and stitch all segments
final_stitched_slice, final_stitched_brightness, final_stitched_stddev, final_stitched_depth, px_spacing_x, px_spacing_y = process_and_stitch_segments(core_structure, mother_dir, 
                                                                                                                                                       width_start_pct=width_start_pct, 
                                                                                                                                                       width_end_pct=width_end_pct, 
                                                                                                                                                       max_value_side_trim=1300,
                                                                                                                                                       min_overlap=10,
                                                                                                                                                       max_overlap=200
                                                                                                                                                       )


# Create csv output directory if it doesn't exist
output_dir_base = "/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/_compiled_logs"
stitched_core_folder = f"{cruise_name}-{core_name}"
output_dir = os.path.join(output_dir_base, stitched_core_folder)
os.makedirs(output_dir, exist_ok=True)

# Display & save final stitched results
display_slice_bt_std(final_stitched_slice,
                    final_stitched_brightness,
                    final_stitched_stddev,
                    pixel_spacing=(px_spacing_x, px_spacing_y),
                    core_name=f"{cruise_name}-{core_name}_CT",
                    save_figs = True,
                    output_dir = output_dir)

# Convert depth from pixels to cm
depth_cm = final_stitched_depth * (total_length_cm / final_stitched_depth[-1])

# Create DataFrame
df = pd.DataFrame({
    'SB_DEPTH_pxl': final_stitched_depth,
    'SB_DEPTH_cm': depth_cm,
    'CT': final_stitched_brightness,
    'CT_std': final_stitched_stddev
})

# Save stitched data into CSV
output_file = os.path.join(output_dir, f'{cruise_name}-{core_name}_CT.csv')
df.to_csv(output_file, index=False)
print(f"CT data saved to: ~/{'/'.join(output_file.split('/')[-3:])}")

<hr>

# **Archived:** Another pattern matching method: Cross-Correlation (not used in stitching CT data)

Cross-correlation measures the similarity between two signals as a function of the displacement of one relative to the other. For two discrete signals $f[n]$ and $g[n]$, the cross-correlation is defined as:

$$(f \star g)[n] = \sum_{m=-\infty}^{\infty} f[m]g[m+n]$$

where $n$ represents the lag or displacement between signals.

In our implementation:
- $f[n]$ represents the y-values of Dataset 1
- $g[n]$ represents the y-values of Dataset 2
- The lag $n$ that maximizes $(f \star g)[n]$ indicates the optimal alignment

## Dataset Stitching Process

1. **Finding Optimal Shift**: 
   - Calculate cross-correlation between datasets
   - Find lag $n_{max}$ that maximizes correlation
   - Convert lag to x-coordinate shift: $x_{shift} = x_1[0] - x_2[0] + n_{max}\Delta x$
   where $\Delta x$ is the x-spacing between points

2. **Overlapping Region Treatment**:
   - Define overlap bounds: $[x_{overlap,min}, x_{overlap,max}]$
   - For overlapping region, final values are computed as:
   $y_{final}(x) = \frac{y_1(x) + y_2(x)}{2}$
   where $y_1(x)$ and $y_2(x)$ are interpolated values from respective datasets

The resulting stitched dataset combines:
- Dataset 1 values for $x < x_{overlap,min}$
- Averaged values for $x_{overlap,min} \leq x \leq x_{overlap,max}$
- Shifted Dataset 2 values for $x > x_{overlap,max}$


In [ ]:
# # Sample Data Creation
# def create_dataset(start, end, num_points=150, noise_std=0.25):
#     """Create a dataset with regular sinusoidal patterns and noise"""
#     x = np.linspace(start, end, num_points)
#     y = (np.sin(x) + 
#          0.5 * np.sin(2*x) + 
#          0.3 * np.sin(3*x) + 
#          np.random.normal(0, noise_std, size=x.shape))
#     return x, y

# # Dataset 1 - First time series
# x1, y1 = create_dataset(0, 10)

# # Dataset 2 - Second time series with phase shift
# x2, y2 = create_dataset(15, 25)

# # Convert to DataFrames for easier manipulation
# df1 = pd.DataFrame({'x': x1, 'y': y1})
# df2 = pd.DataFrame({'x': x2, 'y': y2})

# # Plot Dataset 1
# plt.figure(figsize=(6, 3))
# plt.plot(df1['x'], df1['y'], label='Dataset 1', color='blue')
# plt.title('Dataset 1')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Plot Dataset 2
# plt.figure(figsize=(6, 3))
# plt.plot(df2['x'], df2['y'], label='Dataset 2', color='green')
# plt.title('Dataset 2')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Function to find the largest matching pattern between datasets using dynamic time warping
# def find_largest_pattern_match(df1, df2):
#     """Find the largest matching pattern between two datasets using cross-correlation"""
#     # Get the y values
#     y1 = df1['y'].values
#     y2 = df2['y'].values
    
#     # Calculate cross-correlation
#     correlation = correlate(y1, y2, mode='full')
    
#     # Find the peak correlation and corresponding lag
#     max_corr_idx = np.argmax(correlation)
#     lag = max_corr_idx - (len(y2) - 1)
    
#     # Calculate the optimal shift in x coordinates
#     x_shift = df1['x'].iloc[0] - df2['x'].iloc[0] + lag * (df1['x'].iloc[1] - df1['x'].iloc[0])
    
#     return x_shift, np.max(correlation)

# def stitch_datasets(df1, df2):
#     """Stitch two datasets together using pattern matching"""
#     # Find optimal shift
#     x_shift, correlation_score = find_largest_pattern_match(df1, df2)
    
#     # Apply the shift to dataset 2
#     df2_shifted = df2.copy()
#     df2_shifted['x'] = df2_shifted['x'] + x_shift
    
#     # Find overlapping region
#     x_overlap_min = max(df1['x'].min(), df2_shifted['x'].min())
#     x_overlap_max = min(df1['x'].max(), df2_shifted['x'].max())
    
#     # Create high-resolution x points for overlapping region
#     x_common = np.linspace(x_overlap_min, x_overlap_max, 1000)
    
#     # Interpolate y values
#     y1_interp = np.interp(x_common, df1['x'], df1['y'])
#     y2_interp = np.interp(x_common, df2_shifted['x'], df2_shifted['y'])
    
#     # Weighted average in overlapping region
#     y_avg = (y1_interp + y2_interp) / 2
    
#     # Create separate DataFrames for non-overlapping regions
#     df1_left = df1[df1['x'] < x_overlap_min]
#     df2_right = df2_shifted[df2_shifted['x'] > x_overlap_max]
    
#     # Create DataFrame for overlapping region
#     df_overlap = pd.DataFrame({'x': x_common, 'y': y_avg})
    
#     # Combine all parts to create stitched dataset
#     df_stitched = pd.concat([df1_left, df_overlap, df2_right]).sort_values('x').reset_index(drop=True)
    
#     return df_stitched, x_shift, correlation_score

# # Stitch the datasets
# df_stitched, x_shift, correlation_score = stitch_datasets(df1, df2)

# print(f"Optimal x-shift: {x_shift:.2f}")
# print(f"Correlation score: {correlation_score:.2f}")

# # Plot final result
# plt.figure(figsize=(7, 3))
# plt.plot(df1['x'], df1['y'], label='Dataset 1', color='blue', alpha=0.5)
# plt.plot(df2['x'] + x_shift, df2['y'], label='Dataset 2', color='green', alpha=0.5)
# plt.plot(df_stitched['x'], df_stitched['y'], label='Stitched Dataset', color='red', linewidth=2)
# plt.title('Stitched Dataset with Optimal Pattern Matching')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.legend()
# plt.grid(True)
# plt.show()

# # # Print the stitched data
# # print("\nStitched Data (first 10 rows):")
# # print(df_stitched.head(10))
# # print("\nStitched Data (last 10 rows):")
# # print(df_stitched.tail(10))